In [203]:
import pandas as pd
import numpy as np
from math import ceil
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [204]:
# file location
path1 = '/content/drive/My Drive/cointab/Company X - Order Report.xlsx'
path2 = '/content/drive/My Drive/cointab/Company X - Pincode Zones.xlsx'
path3 = '/content/drive/My Drive/cointab/Company X - SKU Master.xlsx'
path4 = '/content/drive/My Drive/cointab/Courier Company - Invoice.xlsx'
path5 = '/content/drive/My Drive/cointab/Courier Company - Rates.xlsx'

In [223]:
df_Orders = pd.read_excel(path1)
df_pincodes = pd.read_excel(path2)
df_sku = pd.read_excel(path3)
df_courierInvoice = pd.read_excel(path4)
df_courierRates = pd.read_excel(path5)

In [224]:
# CALULATE TOTAL WEIGHT per SKU, COD PRICE
df_Orders_sum = df_Orders.merge(df_sku, on='SKU', how='left')
df_Orders_sum['Net Weight(kg)'] = (df_Orders_sum['Order Qty']*df_Orders_sum['Weight (g)']/1000).round(2)
df_Orders_sum['Net Price'] = df_Orders_sum['Item Price(Per Qty.)']

for i in range(df_Orders_sum.shape[0]):
  df_Orders_sum['Payment Mode'][i] = (0, 1)[ df_Orders_sum['Payment Mode'][i] == 'COD' ] ### mapping PREPAID-COD to 0-1 for calculative convinience

for i in range(df_Orders_sum.shape[0]):
  if df_Orders_sum['Payment Mode'][i]:                                          ### if PAYMENT MODE = 1, i.e COD
    price = df_Orders_sum['Net Price'][i]
    if price > 300:
      df_Orders_sum['Net Price'][i] = 0.05*df_Orders_sum['Net Price'][i]
    else:
      df_Orders_sum['Net Price'][i] = 15
  else:                                                                         ### if PAYMENT MODE = 0, i.e PREPAID
    df_Orders_sum['Net Price'][i] = 0

df_Orders_sum.head()

<ipython-input-224-7cddd85fc583>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Orders_sum['Payment Mode'][i] = (0, 1)[ df_Orders_sum['Payment Mode'][i] == 'COD' ] ### mapping PREPAID-COD to 0-1 for calculative convinience
<ipython-input-224-7cddd85fc583>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Orders_sum['Net Price'][i] = 15
<ipython-input-224-7cddd85fc583>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Orders_

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g),Net Weight(kg),Net Price
0,2001827036,8904223818706,1,1,233,127,0.13,15.0
1,2001827036,8904223819093,1,1,233,150,0.15,15.0
2,2001827036,8904223819109,1,1,233,100,0.10,15.0
3,2001827036,8904223818430,1,1,470,165,0.16,23.5
4,2001827036,8904223819277,1,1,233,350,0.35,15.0


In [225]:
#TOTAL_WEIGHT and COD_CHARGES for each ORDER_ID
### ORDER_ID is the KEY to both the mapping

total_weights = {}
COD_charges = {}

for i in range(df_Orders_sum.shape[0]):
  id = df_Orders_sum['ExternOrderNo'][i]
  total_weights[id] = total_weights.get(id, 0) + df_Orders_sum['Net Weight(kg)'][i]
  COD_charges[id] = COD_charges.get(id, 0) +df_Orders_sum['Net Price'][i]

In [226]:
##### Mapping PINCODES provided by COMPANY X to that of the COURIER COMPANY
pincode_map = {}
for i in range(df_pincodes.shape[0]):
  pincode_map[df_pincodes['Customer Pincode'][i]] = df_pincodes['Zone'][i]
print(len(pincode_map))

df_courierInvoice['Zone by X'] = df_courierInvoice['Customer Pincode'].map(pincode_map)

108


In [227]:
### creating a Collective Order details consisting data from all the tables
df_order_details = pd.DataFrame()
df_order_details[['Order ID', 'AWB Number', 'Zone (Courier Company)', 'Zone (as per X)','Weight(courier comp.)', 'Billing Amount(Rs.)']] = df_courierInvoice[['Order ID', 'AWB Code', 'Zone', 'Zone by X', 'Charged Weight', 'Billing Amount (Rs.)']]

df_order_details['Weight (as per X)'] = df_order_details['Order ID'].map(total_weights)
df_order_details['COD charge'] = df_order_details['Order ID'].map(COD_charges)

df_order_details.head()

,Order ID,AWB Number,Zone (Courier Company),Zone (as per X),Weight(courier comp.),Billing Amount(Rs.),Weight (as per X),COD charge
0,2001806232,1091117222124,d,d,1.30,140.0,1.30,106.95
1,2001806273,1091117222194,d,d,1.00,101.2,0.62,88.25
2,2001806408,1091117222931,d,d,2.50,224.6,2.26,0.00
3,2001806458,1091117223244,b,b,1.00,61.3,0.70,0.00
4,2001807012,1091117229345,d,d,0.15,45.4,0.24,0.00


In [228]:
weight_slabs = {'a':0.25, 'b':0.50, 'c':0.75, 'd':1.25, 'e':1.50}
df_courierRates.set_index('Zone', inplace=True)
df_courierRates.head()

,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
Zone,,,,,
A,0.25,29.5,23.6,13.6,23.6
B,0.50,33.0,28.3,20.5,28.3
C,0.75,40.1,38.9,31.9,38.9
D,1.25,45.4,44.8,41.3,44.8
E,1.50,56.6,55.5,50.7,55.5


In [229]:
### calculate weight slabs for courier company
df_courier_slabs = df_order_details[['Order ID', 'Zone (Courier Company)', 'Weight(courier comp.)']]
df_courier_slabs['num_slabs'] = [0]*len(df_courier_slabs)
df_courier_slabs['weight_slabs_byCourier'] = [0]*len(df_courier_slabs)

for i in range(df_courier_slabs.shape[0]):
  num = ceil(df_courier_slabs['Weight(courier comp.)'][i] / weight_slabs[df_courier_slabs['Zone (Courier Company)'][i]])
  df_courier_slabs['num_slabs'][i] = num
  df_courier_slabs['weight_slabs_byCourier'][i] = num * weight_slabs[df_courier_slabs['Zone (Courier Company)'][i]]


df_courier_slabs.head()

<ipython-input-229-4d6e947d9027>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_courier_slabs['num_slabs'] = [0]*len(df_courier_slabs)
<ipython-input-229-4d6e947d9027>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_courier_slabs['weight_slabs_byCourier'] = [0]*len(df_courier_slabs)
<ipython-input-229-4d6e947d9027>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

,Order ID,Zone (Courier Company),Weight(courier comp.),num_slabs,weight_slabs_byCourier
0,2001806232,d,1.30,2,2.50
1,2001806273,d,1.00,1,1.25
2,2001806408,d,2.50,2,2.50
3,2001806458,b,1.00,2,1.00
4,2001807012,d,0.15,1,1.25


In [230]:
df_expected_charge_calc = df_order_details[['Order ID', 'Zone (as per X)', 'Weight (as per X)', 'COD charge']]
df_expected_charge_calc = df_expected_charge_calc.merge(df_courierInvoice[['Order ID', 'Type of Shipment']], on='Order ID', how='left')

df_expected_charge_calc['num_slabs'] = [0]*len(df_expected_charge_calc)
df_expected_charge_calc['weight_slabs'] = [0]*len(df_expected_charge_calc)
for i in range(df_expected_charge_calc.shape[0]):
  num = ceil(df_expected_charge_calc['Weight (as per X)'][i] / weight_slabs[df_expected_charge_calc['Zone (as per X)'][i]])
  df_expected_charge_calc['num_slabs'][i] = num
  df_expected_charge_calc['weight_slabs'][i] = num * weight_slabs[df_expected_charge_calc['Zone (as per X)'][i]]

df_expected_charge_calc.head()

<ipython-input-230-7fd7b741a105>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expected_charge_calc['num_slabs'][i] = num
<ipython-input-230-7fd7b741a105>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expected_charge_calc['weight_slabs'][i] = num * weight_slabs[df_expected_charge_calc['Zone (as per X)'][i]]


,Order ID,Zone (as per X),Weight (as per X),COD charge,Type of Shipment,num_slabs,weight_slabs
0,2001806232,d,1.30,106.95,Forward charges,2,2.50
1,2001806273,d,0.62,88.25,Forward charges,1,1.25
2,2001806408,d,2.26,0.00,Forward charges,2,2.50
3,2001806458,b,0.70,0.00,Forward charges,2,1.00
4,2001807012,d,0.24,0.00,Forward charges,1,1.25


In [231]:
###calculating total expected fare
forward_charge, rto_charge = [], []
for i in range(df_expected_charge_calc.shape[0]):
  id = df_expected_charge_calc['Zone (as per X)'][i].upper()
  extra_slabs = df_expected_charge_calc['num_slabs'][i]-1

  forward_charge.append(df_courierRates['Forward Fixed Charge'][id] + extra_slabs*df_courierRates['Forward Additional Weight Slab Charge'][id])
  if df_expected_charge_calc['Type of Shipment'][i] == 'Forward charges':
    rto_charge.append(0)

  else:
    rto_charge.append(df_courierRates['RTO Fixed Charge'][id] + extra_slabs*df_courierRates['RTO Additional Weight Slab Charge'][id])

df_expected_charge_calc['forward_charge'] = forward_charge
df_expected_charge_calc['rto_charge'] = rto_charge
df_expected_charge_calc['total_sum'] = df_expected_charge_calc['COD charge'] + df_expected_charge_calc['forward_charge'] + df_expected_charge_calc['rto_charge']
df_expected_charge_calc.head(10)

,Order ID,Zone (as per X),Weight (as per X),COD charge,Type of Shipment,num_slabs,weight_slabs,forward_charge,rto_charge,total_sum
0,2001806232,d,1.30,106.95,Forward charges,2,2.50,90.2,0.0,197.15
1,2001806273,d,0.62,88.25,Forward charges,1,1.25,45.4,0.0,133.65
2,2001806408,d,2.26,0.00,Forward charges,2,2.50,90.2,0.0,90.20
3,2001806458,b,0.70,0.00,Forward charges,2,1.00,61.3,0.0,61.30
4,2001807012,d,0.24,0.00,Forward charges,1,1.25,45.4,0.0,45.40
5,2001806686,d,0.24,30.15,Forward charges,1,1.25,45.4,0.0,75.55
6,2001806885,b,0.84,192.00,Forward charges,2,1.00,61.3,0.0,253.30
7,2001807058,b,1.16,0.00,Forward charges,3,1.50,89.6,0.0,89.60
8,2001807186,d,0.50,0.00,Forward charges,1,1.25,45.4,0.0,45.40
9,2001807290,d,0.50,0.00,Forward charges,1,1.25,45.4,0.0,45.40


In [232]:
df_order_details = df_order_details.merge(df_expected_charge_calc[['Order ID', 'weight_slabs', 'total_sum']], on='Order ID', how='left')
df_order_details.head()

,Order ID,AWB Number,Zone (Courier Company),Zone (as per X),Weight(courier comp.),Billing Amount(Rs.),Weight (as per X),COD charge,weight_slabs,total_sum
0,2001806232,1091117222124,d,d,1.30,140.0,1.30,106.95,2.50,197.15
1,2001806273,1091117222194,d,d,1.00,101.2,0.62,88.25,1.25,133.65
2,2001806408,1091117222931,d,d,2.50,224.6,2.26,0.00,2.50,90.20
3,2001806458,1091117223244,b,b,1.00,61.3,0.70,0.00,1.00,61.30
4,2001807012,1091117229345,d,d,0.15,45.4,0.24,0.00,1.25,45.40


In [233]:
df_order_details = df_order_details.merge(df_courier_slabs[['Order ID', 'weight_slabs_byCourier']], on='Order ID', how='left')
df_order_details.head()

,Order ID,AWB Number,Zone (Courier Company),Zone (as per X),Weight(courier comp.),Billing Amount(Rs.),Weight (as per X),COD charge,weight_slabs,total_sum,weight_slabs_byCourier
0,2001806232,1091117222124,d,d,1.30,140.0,1.30,106.95,2.50,197.15,2.50
1,2001806273,1091117222194,d,d,1.00,101.2,0.62,88.25,1.25,133.65,1.25
2,2001806408,1091117222931,d,d,2.50,224.6,2.26,0.00,2.50,90.20,2.50
3,2001806458,1091117223244,b,b,1.00,61.3,0.70,0.00,1.00,61.30,1.00
4,2001807012,1091117229345,d,d,0.15,45.4,0.24,0.00,1.25,45.40,1.25


In [234]:
df_order_details['Difference'] = df_order_details['total_sum']-df_order_details['Billing Amount(Rs.)']
df_order_details.head()

,Order ID,AWB Number,Zone (Courier Company),Zone (as per X),Weight(courier comp.),Billing Amount(Rs.),Weight (as per X),COD charge,weight_slabs,total_sum,weight_slabs_byCourier,Difference
0,2001806232,1091117222124,d,d,1.30,140.0,1.30,106.95,2.50,197.15,2.50,57.15
1,2001806273,1091117222194,d,d,1.00,101.2,0.62,88.25,1.25,133.65,1.25,32.45
2,2001806408,1091117222931,d,d,2.50,224.6,2.26,0.00,2.50,90.20,2.50,-134.40
3,2001806458,1091117223244,b,b,1.00,61.3,0.70,0.00,1.00,61.30,1.00,0.00
4,2001807012,1091117229345,d,d,0.15,45.4,0.24,0.00,1.25,45.40,1.25,0.00


In [235]:
# df_order_details.drop(columns=['COD charge'], axis=1, inplace=True)
df_order_details = df_order_details.loc[:,['Order ID', 'AWB Number', 'Weight (as per X)', 'weight_slabs', 'Weight(courier comp.)', 'weight_slabs_byCourier',
                          'Zone (as per X)', 'Zone (Courier Company)', 'total_sum', 'Billing Amount(Rs.)', 'Difference']]

df_order_details.head()

,Order ID,AWB Number,Weight (as per X),weight_slabs,Weight(courier comp.),weight_slabs_byCourier,Zone (as per X),Zone (Courier Company),total_sum,Billing Amount(Rs.),Difference
0,2001806232,1091117222124,1.30,2.50,1.30,2.50,d,d,197.15,140.0,57.15
1,2001806273,1091117222194,0.62,1.25,1.00,1.25,d,d,133.65,101.2,32.45
2,2001806408,1091117222931,2.26,2.50,2.50,2.50,d,d,90.20,224.6,-134.40
3,2001806458,1091117223244,0.70,1.00,1.00,1.00,b,b,61.30,61.3,0.00
4,2001807012,1091117229345,0.24,1.25,0.15,1.25,d,d,45.40,45.4,0.00


In [246]:
count_corr, count_over, count_under = 0, 0, 0
amt_corr, amt_over, amt_under = 0, 0, 0
for i in range(df_order_details.shape[0]):
  if df_order_details['Difference'][i] == 0:
    count_corr += 1
    amt_corr += df_order_details['Billing Amount(Rs.)'][i]
  elif df_order_details['Difference'][i] > 0:
    count_under += 1
    amt_under += df_order_details['Difference'][i]
  elif df_order_details['Difference'][i] < 0:
    count_over += 1
    amt_over += df_order_details['Difference'][i]

df_summary = pd.DataFrame()
df_summary[''] = ['Total Orders - Correctly Charged', 'Total Orders - Over Charged', 'Total Order - Under Charged']
df_summary['Count'] = [count_corr, count_over, count_under]
df_summary['Amount'] = [amt_corr, amt_over, amt_under]

df_summary.head()

,,Count,Amount
0,Total Orders - Correctly Charged,10,577.70
1,Total Orders - Over Charged,67,-4171.45
2,Total Order - Under Charged,47,4053.25


In [218]:
df_order_details.columns.values[:] = ['Order ID', 'AWB Number', 'Total weight as per X (KG)', 'Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)', 'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X', 'Delivery Zone charged by Courier Company', 'Expected Charge as per X (Rs.)', 'Charges Billed by Courier Company (Rs.)', 'Difference Between Expected Charges and Billed Charges (Rs.)']
df_order_details.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.30,2.50,1.30,2.50,d,d,197.15,140.0,57.15
1,2001806273,1091117222194,0.62,1.25,1.00,1.25,d,d,133.65,101.2,32.45
2,2001806408,1091117222931,2.26,2.50,2.50,2.50,d,d,90.20,224.6,-134.40
3,2001806458,1091117223244,0.70,1.00,1.00,1.00,b,b,61.30,61.3,0.00
4,2001807012,1091117229345,0.24,1.25,0.15,1.25,d,d,45.40,45.4,0.00


In [222]:
####saving order details to a csv on drive
df_order_details.to_excel('/content/drive/My Drive/cointab/cointab.xlsx', index=False)

In [248]:
df_summary.to_excel('/content/drive/My Drive/cointab/summary.xlsx', index=False)

**PREVIOUS CODE BELOW ⬇**

In [88]:
df_1 = pd.DataFrame()
df_1[['Order ID', 'AWB Number', 'Delivery Zone (Courier Company)','Weight(courier comp.)', 'Billing Amount(Rs.)']] = df_courierInvoice[['Order ID', 'AWB Code', 'Zone', 'Charged Weight', 'Billing Amount (Rs.)']]
df_1.head()

,Order ID,AWB Number,Delivery Zone (Courier Company),Weight(courier comp.),Billing Amount(Rs.)
0,2001806232,1091117222124,d,1.30,140.0
1,2001806273,1091117222194,d,1.00,101.2
2,2001806408,1091117222931,d,2.50,224.6
3,2001806458,1091117223244,b,1.00,61.3
4,2001807012,1091117229345,d,0.15,45.4


In [89]:
df_order_weight = df_Orders.merge(df_sku, on='SKU', how='left')
df_order_weight.head(10)

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.),Weight (g)
0,2001827036,8904223818706,1,COD,233,127
1,2001827036,8904223819093,1,COD,233,150
2,2001827036,8904223819109,1,COD,233,100
3,2001827036,8904223818430,1,COD,470,165
4,2001827036,8904223819277,1,COD,233,350
5,2001827036,GIFTBOX202002,1,COD,659,500
6,2001827036,GIFTBOX202002,1,COD,659,500
7,2001827036,8904223818638,2,COD,570,137
8,2001827036,SACHETS001,1,COD,350,10
9,2001825261,8904223819024,4,Prepaid,3636,112


In [90]:
#total weight for each order id
total_weights = {}
for i in range(df_order_weight.shape[0]):
  id = df_order_weight['ExternOrderNo'][i]
  total_weights[id] = total_weights.get(id, 0) + df_order_weight['Weight (g)'][i]

print(len(total_weights))

124


In [91]:
df_1['Weight(by X)(g)'] = df_1['Order ID'].map(total_weights)
df_1.tail(15)

,Order ID,AWB Number,Delivery Zone (Courier Company),Weight(courier comp.),Billing Amount(Rs.),Weight(by X)(g)
109,2001809270,1091117613962,d,0.68,95.2,500
110,2001809934,1091117803511,d,0.82,98.2,361
111,2001810125,1091117804314,d,0.66,90.2,500
112,2001810281,1091117805390,d,0.68,90.2,500
113,2001810549,1091117806263,d,1.86,191.8,849
114,2001810697,1091117807140,d,2.27,224.6,607
115,2001811039,1091117904860,d,0.68,90.2,488
116,2001811058,1091117905022,d,0.72,90.2,500
117,2001811306,1091117958163,d,1.10,135.0,945
118,2001812195,1091118442390,d,0.67,90.2,500


In [93]:
df_1['Weight(by X)(g)'] = df_1['Weight(by X)(g)']/1000
df_1.head()

,Order ID,AWB Number,Delivery Zone (Courier Company),Weight(courier comp.),Billing Amount(Rs.),Weight(by X)(g)
0,2001806232,1091117222124,d,1.30,140.0,0.000377
1,2001806273,1091117222194,d,1.00,101.2,0.000615
2,2001806408,1091117222931,d,2.50,224.6,0.000929
3,2001806458,1091117223244,b,1.00,61.3,0.000700
4,2001807012,1091117229345,d,0.15,45.4,0.000240


In [94]:
pincode_map = {}
for i in range(df_pincodes.shape[0]):
  pincode_map[df_pincodes['Customer Pincode'][i]] = df_pincodes['Zone'][i]
print(len(pincode_map))

108


In [114]:
df_courierInvoice['Zone by X'] = df_courierInvoice['Customer Pincode'].map(pincode_map)

In [96]:
df_1['Zone(as per X)'] = df_courierInvoice['Zone by X']
df_1.head(15)

,Order ID,AWB Number,Delivery Zone (Courier Company),Weight(courier comp.),Billing Amount(Rs.),Weight(by X)(g),Zone(as per X)
0,2001806232,1091117222124,d,1.30,140.0,0.000377,d
1,2001806273,1091117222194,d,1.00,101.2,0.000615,d
2,2001806408,1091117222931,d,2.50,224.6,0.000929,d
3,2001806458,1091117223244,b,1.00,61.3,0.000700,b
4,2001807012,1091117229345,d,0.15,45.4,0.000240,d
5,2001806686,1091117229555,d,0.15,45.4,0.000240,d
6,2001806885,1091117229776,b,1.00,61.3,0.000420,b
7,2001807058,1091117323112,b,1.15,89.6,0.001168,b
8,2001807186,1091117323812,d,0.50,45.4,0.000500,d
9,2001807290,1091117324206,d,0.50,45.4,0.000500,d


In [97]:
df_courierRates.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.25,29.5,23.6,13.6,23.6
1,B,0.50,33.0,28.3,20.5,28.3
2,C,0.75,40.1,38.9,31.9,38.9
3,D,1.25,45.4,44.8,41.3,44.8
4,E,1.50,56.6,55.5,50.7,55.5


In [98]:
df_Orders.head()

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233
